# GUIDE
- load_data() 시 경고 뜨는 것.
-  huggingface_hub 라이브러리가 캐시 시스템에서 심볼릭 링크(symlink)를 사용하려 할 때, 현재 시스템에서 이를 지원하지 않아 발생합니다. 심볼릭 링크는 동일한 파일을 효율적으로 저장하기 위해 사용되며, 이를 통해 디스크 공간을 절약할 수 있습니다. 그러나 Windows에서는 기본적으로 심볼릭 링크 생성이 제한되어 있어, 이를 활성화하지 않으면 캐시 시스템이 비효율적으로 작동하여 더 많은 디스크 공간을 사용할 수 있습니다.

In [ ]:
# !uv pip install datasets transformers scikit-learn evaluate torchinfo tqdm accelerate

In [1]:
!uv pip install datasets

Audited 1 package in 279ms


# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.

![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

# HuggingFace Datasets 패키지

- HuggingFace Datasets는 머신러닝 모델 학습에 필요한 대규모 데이터셋을 효율적으로 로드, 전처리, 변환, 스트리밍할 수 있도록 지원하는 라이브러리이다.
- HuggingFace Hub에 등록된 다양한 데이터셋을 동일한 인터페이스로 접근할 수 있다.
- 설치

  - `pip install datasets`
- 공식 문서

  - [https://huggingface.co/datasets](https://huggingface.co/datasets)
  - [https://github.com/huggingface/datasets](https://github.com/huggingface/datasets)

## Dataset과 DatasetDict

### Dataset

- 하나의 데이터셋을 테이블 형태(행 = 샘플, 열 = Feature)로 저장하는 구조.
- 텍스트, 이미지, 오디오, 멀티모달 데이터 지원.
- PyTorch, TensorFlow 학습 파이프라인과 연동 가능.
- Lazy Loading 및 Streaming을 통해 대규모 데이터도 메모리 효율적으로 처리.

### DatasetDict

- 여러 Dataset을 key-value 형태로 묶어 관리한다.

  - 예: `{"train": Dataset, "validation": Dataset, "test": Dataset}`
- key를 통해 데이터셋에 접근.

## DatasetDict / Dataset 로딩
- `datasets.load_dataset()` 이용
  
### Hub 데이터 로딩
- Huggingface hub에 등록된 Dataset의 id를 전달하면  다운로드 후 로딩한다.

```python
from datasets import load_dataset
dataset_dict = load_dataset("imdb")
```
![img](figures/huggingface_dataset.png)

### 로컬 파일 로딩

```python
dataset_dict = load_dataset(
    "csv",
    data_files={
        "train": "data/train.csv",
        "validation": "data/valid.csv",
        "test": "data/test.csv"
    }
)
```

### 다양한 소스에서 Dataset 생성

```python
from datasets import Dataset

Dataset.from_pandas(df) # DataFrame을 Dataset으로 로딩
Dataset.from_dict({"text": [...], "label": [...]}) # 딕셔너리를 Dataset으로 로딩
Dataset.from_csv("path.csv") # 경로의 csv파일을 Dataset으로 로딩
```

## Dataset 주요 기능

### 데이터 접근(조회)

- 인덱싱으로 샘플 조회: `dataset[0]`
- 슬라이싱: `dataset[:10]`

### 전처리 관련 메소드 상세 설명

#### map()

데이터셋의 모든 데이터(또는 지정한 batch)에 변환 함수를 적용한다.

```python
dataset.map(
    function,                    # 각 샘플/배치에 적용할 함수
    batched=False,               # True: 배치 단위 입력
    batch_size=1000,             # batched=True일 때 배치사이즈. 기본값: 1000
    remove_columns=None,         # 변환 후 제거할 컬럼 리스트
    num_proc=None,               # 병렬 처리 프로세스 개수
)
```

##### map에 전달하는 함수를 정의할 때 입력(parameter)/출력(return) 형태

- `batched=False` (기본값) 일 때

  - 함수 입력: dict (하나의 row)
  - 함수 반환: dict (추가될 컬럼 포함)

```python
def add_length(example):
    example["text_len"] = len(example["text"])
    return example

dataset = dataset.map(add_length)
```

- `batched=True` 일때 (배치단위 입력)

  - 함수 입력: dict (여러 row의 컬럼이 리스트로 묶여 들어옴)
  - 함수 반환: dict (리스트 형태 유지)

```python
def add_length_batch(batch):
    return {"text_len": [len(t) for t in batch["text"]]}

dataset = dataset.map(add_length_batch, batched=True)
```

#### filter()

조건 함수를 적용하여 True를 반환하는 row만 남긴다.

```python
dataset.filter(
    function,           # 조건 함수
    batched=False,      # 배치 처리 여부
    num_proc=None,      # 병렬 처리
    desc=None
)
```

##### filter()에 전달하는 함수를 정의할 때 입력(parameter)/출력(return) 형태

- `batched=False`: 한 줄씩 조건 적용

```python
dataset = dataset.filter(lambda x: x["label"] == 1)
```

- `batched=True`: 배치 활용

```python
def filter_positive(batch):
    return [label == 1 for label in batch["label"]]

dataset = dataset.filter(filter_positive, batched=True)
```

## 기타 유용 메소드

- **컬럼 삭제**
  `dataset = dataset.remove_columns(["column_name"])`
- **컬럼 이름 변경**
  `dataset = dataset.rename_column("orig", "new")`
- **데이터 분할**
  `dataset.train_test_split(test_size=0.2)`

- **저장**

```python
dataset.save_to_disk("path/")
dataset = datasets.load_from_disk("path/")
```

- 모델학습 프레임워크용 포맷 변환
  - 기본적으로 Dataset의 데이터들은 list 로 저장됨.
  - pytorch나 tensorflow에서 사용할 수있도록 format을 변환.

```python
dataset.set_format(type="torch") # numpy, torch, tensorflow, python
```

- Streaming 로딩
    - Huggingface Hub의 데이터셋이 너무 클 경우(수백 GB) 필요한 데이터만 순차적으로 가져와 로딩하여 처리할 수있다.
```python
ds_stream = load_dataset("c4", split="train", streaming=True)
```


# 유튜브 댓글 감정 분석 데이터셋
- https://huggingface.co/datasets/LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset
- `LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset`

In [1]:
from datasets import load_dataset

#Huggingface dataset hub의 Dataset 로딩 -> ID
dataset_dict = load_dataset("LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset")


In [3]:
type(dataset_dict)

datasets.dataset_dict.DatasetDict

In [4]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['cid', 'text', 'time', 'author', 'channel', 'votes', 'replies', 'heart', 'reply', 'time_parsed', 'source_video_id', 'source_video_title', 'text_length', 'is_not_spam', 'votes_norm', 'replies_norm', 'heart_norm', '검수_감정'],
        num_rows: 10035
    })
})

In [2]:
#DatasetDict에서 Dataset 조회
dataset = dataset_dict['train']
type(dataset)

datasets.arrow_dataset.Dataset

In [6]:
dataset

Dataset({
    features: ['cid', 'text', 'time', 'author', 'channel', 'votes', 'replies', 'heart', 'reply', 'time_parsed', 'source_video_id', 'source_video_title', 'text_length', 'is_not_spam', 'votes_norm', 'replies_norm', 'heart_norm', '검수_감정'],
    num_rows: 10035
})

In [3]:
#Dataset 관련정보조회
#feature(컬럼) 정보

dataset.features #컬럼명: 컬럼의 타입

{'cid': Value('string'),
 'text': Value('string'),
 'time': Value('string'),
 'author': Value('string'),
 'channel': Value('string'),
 'votes': Value('int64'),
 'replies': Value('int64'),
 'heart': Value('bool'),
 'reply': Value('bool'),
 'time_parsed': Value('float64'),
 'source_video_id': Value('string'),
 'source_video_title': Value('string'),
 'text_length': Value('int64'),
 'is_not_spam': Value('bool'),
 'votes_norm': Value('float64'),
 'replies_norm': Value('float64'),
 'heart_norm': Value('int64'),
 '검수_감정': Value('string')}

In [8]:
dataset.num_columns # feature의 개수
dataset.num_rows # 행의 개수 -> 총 데이터 수

10035

In [4]:
## 컬럼을 제거(삭제)
dataset = dataset.remove_columns(['cid', 'time', 'author', 'channel', 'votes', 'replies', 'heart', 'reply', 'time_parsed', 'source_video_id', 'source_video_title', 'text_length', 'is_not_spam', 'votes_norm', 'replies_norm', 'heart_norm'])

dataset

Dataset({
    features: ['text', '검수_감정'],
    num_rows: 10035
})

In [5]:
## feature(컬럼) 이름 변경. dict로 입력 - {현재이름 : 변경할 이름}
dataset = dataset.rename_columns({
    "text":"comment", 
    "검수_감정":"labels"
})
dataset

Dataset({
    features: ['comment', 'labels'],
    num_rows: 10035
})

In [ ]:
#데이터 셋의 값을 조회
dataset[0]
dataset[:3]
dataset[[1, 4, 9]]

{'comment': ['팬덤이 계속 보길 원하고, 글로벌적 수요가 분명 타레이블보다 높음에도 경영진의 입맛에 맞지 않다는 이유로 공연을 막고, 광고계약을 막아서 수입창출을 막는 것은 부당한 것 맞음. 법이 제대로 해석해내지못한 부분이 분명 있음\n\n법이 불완전한 부분이 있는데, 부당해도 법이 이러니깐 감내해. 너희가 뭔데 나대는데.. 이런식의 비야냥과 무력한 인간들의 조롱조차도 여태해왔던 용기와 에너지로 이겨내요, PERSON_003.\n일일히 댓글달지 못하지만, 마음으로 응원하고 당신들의 목소리에 귀기울이고 이해하고 있는 사람은 그대들이 생각하는 것보다 훨씬 많아요.\n세상을 바꿀지도 모를 그대들을 응원합니다.  \n 마흔의 나이에도 어린 당신들의 용기를 배웁니다.',
  '해외인터뷰에 혐한발언은 진짜 미친거아님? 왜 가면갈수록 멍청한 발언만 골라가면서 하냐? 멤버들은 어리니까 그렇다쳐도 케어해주는 부모들은 뭔생각인거지??',
  '와 미춌다!! 울 PERSON_004 넘 이뻐!ㅜㅠ💙🩷💛💚💜🐰🐰🐰🐰🐰'],
 'labels': ['긍정', '부정', '긍정']}

In [13]:
# feature의 값들 조회 -indexing/slicing
dataset['comment'][0]
dataset['comment'][:5]

['PERSON_002 예쁘고 귀엽다 언제든지 PERSON_001 응원하고 있을게 화이팅!!!!',
 '팬덤이 계속 보길 원하고, 글로벌적 수요가 분명 타레이블보다 높음에도 경영진의 입맛에 맞지 않다는 이유로 공연을 막고, 광고계약을 막아서 수입창출을 막는 것은 부당한 것 맞음. 법이 제대로 해석해내지못한 부분이 분명 있음\n\n법이 불완전한 부분이 있는데, 부당해도 법이 이러니깐 감내해. 너희가 뭔데 나대는데.. 이런식의 비야냥과 무력한 인간들의 조롱조차도 여태해왔던 용기와 에너지로 이겨내요, PERSON_003.\n일일히 댓글달지 못하지만, 마음으로 응원하고 당신들의 목소리에 귀기울이고 이해하고 있는 사람은 그대들이 생각하는 것보다 훨씬 많아요.\n세상을 바꿀지도 모를 그대들을 응원합니다.  \n 마흔의 나이에도 어린 당신들의 용기를 배웁니다.',
 'PERSON_003 데뷔때부터 팬인데 이젠 나도 쉴드못쳐주겟다.. 하니가 국회 갔을때부터 일이 ㅈㄴ 잘못됐어 ㅆ1ㅂ; 그이후로 본인들이 감당 못할말을 계속 해버리니까 이사단이 나지 진짜 개슬프네..PERSON_003',
 '참고로 PERSON_003 탄핵집회를 선결제해줬다 ㅋㅋ',
 '해외인터뷰에 혐한발언은 진짜 미친거아님? 왜 가면갈수록 멍청한 발언만 골라가면서 하냐? 멤버들은 어리니까 그렇다쳐도 케어해주는 부모들은 뭔생각인거지??']

In [14]:
dataset['labels'][:10]

['긍정', '긍정', '부정', '건너뛰기', '부정', '중립', '긍정', '부정', '부정', '긍정']

In [7]:
# feature 의 고유값(unique 값) 조회
dataset.unique("labels")

['긍정', '부정', '건너뛰기', '중립', '불명확', '', '긍정정', '중립립']

In [ ]:
# "긍정정" -> "긍정" , "중립립" -> "중립", "건너뛰기"-> "불명확"(차후에 삭제)
# 일괄처리
# map : 전체 값을 받아서 일괄적으로 처리하는 함수

label_map = {
    "긍정정":"긍정",
    "중립립":"중립",
    "건너뛰기":"불명확"
}

def update_labels(row):
    # row : map이 전달할 dataset의 한 개 행 값. {"comment":"댓글", "labels":"라벨"}
    # map(batched=True)일때: {"comment":[배치개수], "labels":[ㅠㅐ치개수]}
    label = row['labels']
    new_label = label_map.get(label, label) 
    # label_map.get(label, label) label_map에서 설정한 바뀔 내용은 바뀌고, 바꿔야 할 항목이 설정되지 않았으면 그대로 둠
    return {"labels":new_label} # {"feature 이름": 처리결과}

dataset = dataset.map(update_labels) #, batched=True) : 한번에 여러개씩 처리. 디폴트 : 100

# map(함수) : 일괄 처리
# 함수(개별행): {처리결과를 저장할 feature 이름 : 처리결과}

Map:   0%|          | 0/10035 [00:00<?, ? examples/s]

In [9]:
dataset.unique('labels')

['긍정', '부정', '불명확', '중립', '']

In [ ]:
# 불명확 , '' 인 labels 제거 - > 행을 제거
# 특정 라벨들의 정보만 불러올 수도 있나?
# filter()를 이용해서 labels가 "불명확", "" 이 아닌 행들만남긴다.
# filter(함수) -> dataset의 개별행을 받아서 bool을 반환. 그 행을 True 면 남기고 False면 삭제

def check_labels(row):
    #filter에 전달할 함수.
    #파라미터: Dataset의 한 행(batched=True - 여러행)
    #         dict - {컬럼명:값, 컬럼명:값}, batched=True: {컬럼명:[배치개수]}
    # return: bool - 그 행을 사용할지 여부
    flag = row['labels'] != '불명확' and row ['labels'] != ''
    return flag

dataset = dataset.filter(check_labels)
#dataset = dataset.filter(lambda row : row['labels'] != '불명확' and row ['labels'] != '')

Filter:   0%|          | 0/10035 [00:00<?, ? examples/s]

In [11]:
dataset

Dataset({
    features: ['comment', 'labels'],
    num_rows: 5571
})

In [12]:
dataset.unique('labels')

Flattening the indices:   0%|          | 0/5571 [00:00<?, ? examples/s]

['긍정', '부정', '중립']

## Train/Test set split

In [ ]:
dataset.features
# Value(타입): 행마다 제각각 다른 값들을 가진다.(범주헝이 아니다.)
# ClassLabel: 범주형 컬럼 - 문자열 범주값 - 정수범주값 으로 관리

{'comment': Value('string'), 'labels': Value('string')}

In [16]:
#labels의 타입을 ClassLabel로 변환
dataset=dataset.class_encode_column("labels")

In [17]:
dataset.features

{'comment': Value('string'), 'labels': ClassLabel(names=['긍정', '부정', '중립'])}

In [23]:
#ClassLabel 타입 featuer 사용
dataset['labels'].features
dataset['labels'].features.names # class 이름(문자열), index:class id(정수)
dataset['labels'].features.str2int('긍정') # class 이름으로 class id 조회
dataset['labels'].features.int2str(2) # class id로 class 이름 조회

'중립'

In [24]:
## 섞고 나서 분할
# 섞기
dataset = dataset.shuffle() # seed=42 seed : random seed 값 지정 -> 항상 일정한 순서로

#분할
dataset_dict = dataset.train_test_split( # 사이킷런과 동일한 이름의 메소드
    test_size=0.15, #test set 의 비율
    stratify_by_column="labels"
    # 지정한 컬럼의 값들의 비율에 맞게 분할. 지정한 feauture는 ClassLabel(범주형) 타입이여야함
) 


In [ ]:
dataset_dict # 자동으로 이름이 붙음

DatasetDict({
    train: Dataset({
        features: ['comment', 'labels'],
        num_rows: 4735
    })
    test: Dataset({
        features: ['comment', 'labels'],
        num_rows: 836
    })
})

In [26]:
train_set = dataset_dict['train']
test_set = dataset_dict['test']

In [27]:
train_set
test_set

Dataset({
    features: ['comment', 'labels'],
    num_rows: 836
})

In [29]:
# 전처리한 Dataset을 huggingface datahub에 upload
# 1. login -> access token
# 2. upload: dataset/dataset_dict.push_to_hub("계정/dataset_id")

#로그인

from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv() # .env의 key velue 들이 os 의 환경변후로 저장.
hf_token = os.getenv("HUGGINGFACE_API_KEY")

login(hf_token)


In [ ]:
dataset_dict.push_to_hub("beth9165/youtube_comment_sentiment_dataset_preprocessing")
# git 기반이기 때문에 수정 사항이 있으면 다시 로드 하기도 함.

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/beth9165/youtube_comment_sentiment_dataset_preprocessing/commit/c5e68dc76e5e4f15aaff53264914f5a6bac5be2b', commit_message='Upload dataset', commit_description='', oid='c5e68dc76e5e4f15aaff53264914f5a6bac5be2b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/beth9165/youtube_comment_sentiment_dataset_preprocessing', endpoint='https://huggingface.co', repo_type='dataset', repo_id='beth9165/youtube_comment_sentiment_dataset_preprocessing'), pr_revision=None, pr_num=None)

In [31]:
# huggingface에 업로드 한 데이터 셋 LOAD
from datasets import load_dataset
load_datasetdict = load_dataset("beth9165/youtube_comment_sentiment_dataset_preprocessing")

README.md:   0%|          | 0.00/662 [00:00<?, ?B/s]

c:\Users\USER\Documents\SKN21\09_Huggingface_transformers\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\datasets--beth9165--youtube_comment_sentiment_dataset_preprocessing. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/364k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4735 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/836 [00:00<?, ? examples/s]

In [32]:
##########################################
# 로컬 컴퓨터에 저장
##########################################
dataset_dict.save_to_disk("data/youtube_comment_dataset") # 저장할 디렉토리 경로

Saving the dataset (0/1 shards):   0%|          | 0/4735 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/836 [00:00<?, ? examples/s]

In [34]:
# 불러오기
from datasets import load_from_disk
load_dataset_dict2 = load_from_disk("data/youtube_comment_dataset")
load_dataset_dict2


DatasetDict({
    train: Dataset({
        features: ['comment', 'labels'],
        num_rows: 4735
    })
    test: Dataset({
        features: ['comment', 'labels'],
        num_rows: 836
    })
})

In [35]:
########################
# Dataset을 다른 형식으로 변환
train_set = dataset_dict['train']
# Dataset을 pandas DataFrame
df_train = train_set.to_pandas()
df_train.head()

,comment,labels
0,라엘이 어머니인거진심으로축하드려요ㅋ라엘이도 넘귀욤,0
1,하.... 얘들아... 어떤식으로든 돌아와줬으면 좋겠다 좋은 프로듀서와 좋은 회사도...,0
2,근데 아이유님보다 지드래곤님이 더 선배잖아요 가요계는 일반 직장보다 더 위계질서가 ...,1
3,너네가 저절로 큰거같지? 피프티랑 똑같다,1
4,우와 진짜 재밌어요 앞으로 건강하세요,0


In [ ]:
# train_set.to_dict()

In [37]:
# 다른 형식을 Dataset으로 - Dataset.from_xxxxx()
from datasets import Dataset 

d = Dataset.from_pandas(df_train.head(100))
d

Dataset({
    features: ['comment', 'labels'],
    num_rows: 100
})

## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "beomi/kcbert-base" # 한국어 학습한 BERT base model - feature 추출이 목적
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, # FE
    num_labels=3  # 분류할 클래스 개수
)
# Feature Extractor: Pretrained(사전학습) 모델
# 추론기 : 학습되지 않은 모델
#Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# 경고 : 학습되지 않은 모델이라는 의미

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [41]:
!uv pip install torchinfo

Resolved 1 package in 76ms
Installed 1 package in 42ms
 + torchinfo==1.8.0


In [42]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   23,040,000
│    │    └─Embedding: 3-2                                   230,400
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

In [ ]:
################################
# trainset/testset 을 학습용으로 변환
# 현재 : comment(string), labels(범주형-ClassLabel)
# 변환 : 입력(comment) => 토큰화 "input_ids", "token_type_ids", "attention_mask", labels

In [ ]:
r = tokenizer("안녕하세요", return_tensors="pt")
r
model(**r)
# model("안녕하세요") # type error 모델에는 텐서로 데이터가 입력되어야 함.

In [43]:
#dataset.map()
train_set = load_datasetdict['train']
test_set = load_datasetdict['test']

In [ ]:
def tokenize_comment(rows):
    # batch 단위로 행들을 받아 comment들을 tokenize 해서 반환.
    # rows :  {"comment": [댓글, 댓글, ..], "labels":[라벨, 라벨, ...]}
    return tokenizer(rows['comment'], truncation=True) # list type 

trainset = train_set.map(
    tokenize_comment,
    batched=True,
    # batch_size=200 > 설정된 배치 사이즈 값 만큼 들어감. 디폴트는 1000 개, batched = False 인 경우는 하나씩
    remove_columns=['comment'] # 토큰화 데이터가 완성되고 난 후, 지정한 컬럼을 dataset 에서 제거

)
# feature : comment, labels, imput_ids, token_type_ids, attention_mask

testset = test_set.map(
    tokenize_comment,
    batched=True,
    remove_columns=['comment']
)

trainset.set_format(type="pt")
testset.set_format(type="pt")
# 각 데이터들을 조회하면 반환할 때 지정 한 타입으로 변환해서 반환 (pt:torch.tensor)

Map:   0%|          | 0/4735 [00:00<?, ? examples/s]

Map:   0%|          | 0/836 [00:00<?, ? examples/s]

In [48]:
trainset[0]

{'labels': tensor(0),
 'input_ids': tensor([    2,  1114,  4390,  4017, 13244,  8926,  4153, 17054,  4728,  4159,
         20201,  4157,  4007,  4390,  9905,   645,  4115,  5156,     3]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

## 학습 (Hugging Face Transformers)

- Transformers 라이브러리는 모델 학습을 위해 `TrainingArguments`, `Trainer` 클래스를 제공한다.
- `Trainer`:
  -  데이터 로딩, 학습 루프, 평가, 로깅, 모델 저장 등을 자동으로 처리하여, 복잡한 학습 절차를 단순한 코드로 구성할 수 있도록 돕는다.
- `TrainingArguments`:
  -  `Trainer`가 사용할 **학습 관련 설정**(학습 파라미터, 로그/체크포인트 저장 방식 등)을 담당하는 클래스이다.
- 이들을 활용하면 **학습 옵션 설정, 로깅, Gradient Accumulation, Mixed Precision(AMP) 적용, 평가 및 체크포인트 관리** 등을 손쉽게 수행할 수 있다.
- `Trainer`를 사용하려면 **Datasets** 라이브러리 또는 `torch.utils.data.Dataset` 형식의 데이터셋이 필요하다.
- 일반적으로는 `Trainer`만 써도 파인튜닝을 빠르고 편리하게 할 수 있다. 하지만 학습 과정에서 특별한 계산을 해야 하거나, 일반적이지 않은 방식으로 학습을 진행해야 할 때는 `Trainer`를 쓰기 어렵다. 이럴 때는 직접 `PyTorch`로 학습 루프를 작성해 원하는 방식대로 조정한다.


In [51]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
hf_token = os.getenv("HUGGINGFACE_API_KEY")


In [ ]:
from transformers import TrainingArguments, Trainer

# 학습된 모델을 Huggingface-hub 에 올릴 때 사용할 model_id
hub_model_id = "beth9165/youtube_comment_sentiment_kcbert_finetune"

train_args = TrainingArguments(
    output_dir="saved_models", #학습 도중 저장할 디렉토리
    #fp16=True #weight의 타입을 float16으로 조정. 메모리가 부족할 경우 지정(float32가 기본)
    
    logging_strategy="epoch", # train loss 에폭단위로 출력
    
    num_train_epochs=5, # 학습 에폭 수
    per_device_train_batch_size=16, # 학습 시 batch size
    per_device_eval_batch_size=64, # 검증 평가시 batch size
    
    # 지정한 step 만큼 순전파한 것을 모아서 한 번에 파라미터 업데이트
    # memory 문제로 batch size를 크게 할 수 없을 때 설정
    # batch_size를 per_device_train_batch_size * gradient_accumulation_steps 로 지정한 효과
    gradient_accumulation_steps=4,

    # 학습 도중 모델을 평가/저장 할 때 epoch/step  중 어떤 기준으로 할 지. "epoch" "step" "no"
    eval_strategy='epoch', 
    save_strategy='epoch', 

    # 학습이 끝나고 나서 학습 도중 가장 성능이 좋았던 epoch의 weight로 모델의 weight를 변경
    ## Best 모델을 저장/hf-hub 에 올릴 때, True로 설정. # 어차피 제일 좋은 모델로 해야하는데. 귀찮게 설정하게 만들었네.
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss", # best modell 선정시 확인 할 성능지표
    greater_is_better=False, # 성능이 높은게 좋은 건지 낮은게 좋은 건지 설정. # loss는 낮을 수록 좋음. acc는 높을 수록 좋음.

    # hf-hub 올리기 위한 설정 - 학습 완료 후 자동으로 commit 하기
    push_to_hub=True, # upload 할 지 여부
    hub_model_id=hub_model_id, #hub에 올릴 모델 ID
    hub_token=hf_token, # HF Access Token

    report_to="none"

)

### 평가함수 정의
- `evaluate` 패키지를 이용해 정의한다.
- `evaluate` 패키지
  - evaluate 패키지는 **모델 성능을 측정하기 위한 평가 지표(Metrics)**를 손쉽게 불러오고 사용할 수 있도록 하는 라이브러리이다.
  - 다양한 머신러닝 분야에서 사용되는 평가지표를 지원하며 필요하면 사용저 정의 평가지표를 정의할 수도 있다.

In [ ]:
# !uv pip install evaluate scikit-learn

Resolved 41 packages in 145ms
Prepared 1 package in 1.78s
Installed 4 packages in 554ms
 + joblib==1.5.2
 + scikit-learn==1.8.0
 + scipy==1.16.3
 + threadpoolctl==3.6.0


In [ ]:
# - 정확도 예
import evaluate
import torch

acc_fn = evaluate.load("accuracy") # 평가 함수를 load
dummy_pred = torch.tensor([1,0,0,1,0])
dummy_y = torch.tensor([0,0,1,0,1])

#평가
result = acc_fn.compute(
    predictions=dummy_pred, # 모델 예측 label
    references=dummy_y      # 정답 label
)

result

{'accuracy': 0.2}

In [59]:
# 학습 도중/완료 후 모델을 평가할 때 사용할 함수 정의
## 기본적으로 loss 는 계산을 한다. 그 이외 평가지표로 평가를 할 경우 정의하고 Trainer 에 전달.

def compute_metrics(pred):
    """
    정확도, F1 score를 평가.
    Args:
        pred(EvalPrediction) - 모델 예측값, 정답을 묶어서 제공.
    Retrun:
        dict - key: 평가지표 이름, value:평가 점수
    """
    labels = pred.label_ids # EvalPrediction의 정답(label-[1,2,0,1, ...])
    # 모델의 출력값 [[0 확률, 1확률, 2확률 -logit], [...]] -> class 값으로 변환
    preds = pred.predictions.argmax(-1)

    # 평가함수를 로딩
    acc_fn = evaluate.load("accuacy")
    f1_fn = evaluate.load("f1")
    #평가
    acc = acc_fn.compute(predictions=preds, references=labels)
    f1 = f1_fn.compute(predictions=preds, references=labels, average="weighted")

    return {"accuracy":acc, "f1-score":f1}

### Collator 함수

- **Collator(Data Collator)**는 **데이터셋에서 샘플 단위(raw examples)를 꺼낸 뒤 → 하나의 배치(batch)로 묶을 때 필요한 전처리를 수행하는 함수**이다.
- 즉, DataLoader가 `batch_size=8`로 8개의 샘플을 가져왔다면, collator는 이 8개의 샘플을 **모델이 바로 입력할 수 있는 형태**로 변환한다.
- Transformer 모델은 각 샘플의 길이가 다르므로, 이들을 자동으로 **패딩(padding)**·**마스크 생성**·**텐서 변환** 등을 해주는 장치가 필요하며, 이 역할을 담당하는 것이 **collator**이다.

#### collator가 수행하는 핵심 역할
- **정적 패딩(Static padding)/동적 패딩(Dynamic Padding)**
  - 모든 문장의 길이를 지정한 길이에 맞추기 위해 길이가 적은 경우 패딩 토큰을 추가한다.
  - 모든 학습 데이터셋의 길이를 동일하게 맞추는 것을 정적패딩, 배치 단위로 맞추는 것을 동적 패딩이라고 한다.
  - 전체 데이터셋을 미리 패딩하는 정적 패딩보다 **동적 패딩**이 메모리를 절약하고 속도를 증가시켜 더 효율적이다.

- **Attention Mask 생성**
  - 패딩된 부분은 계산에서 제외해야 한다. collator는 `attention_mask`를 자동으로 생성해 준다.

```
Input Ids:     [101, 2003, 2023, 102, 0,   0]
Attention Mask:[ 1,   1,   1,   1, 0,   0]
```

- **Label 패딩 / 정렬**

  - 텍스트 생성/요약 등에서는 target sequence도 padding 필요
  - BERT의 MLM(Masked LM)에서는 label에서 패딩에 -100을 넣는 등 규칙이 있음  (PyTorch CrossEntropyLoss는 -100을 무시)

- **텐서 변환**

    - 리스트(list) 형태의 샘플을 PyTorch/TensorFlow 텐서로 변환한다.

#### Hugging Face에서 제공하는 대표 Collator들

| Collator                             | 주 사용 모델         | 역할                      |
| ------------------------------------ | --------------- | ----------------------- |
| `DataCollatorWithPadding`            | 대부분의 모델         | 동적 패딩 처리                |
| `DataCollatorForLanguageModeling`    | BERT, RoBERTa 등 | MLM mask 생성 및 label 생성  |
| `DataCollatorForSeq2Seq`             | T5, BART 등      | 입력/출력 둘 다 패딩 및 label 정리 |
| `DataCollatorForTokenClassification` | NER 등           | token-level label 패딩    |
| `DefaultDataCollator`                | 간단한 task        | 미리 패딩된 입력만 병합           |

#### 예시

```python
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader 예시
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=4, collate_fn=collator)
```


In [60]:
from transformers import DataCollatorWithPadding

#Batch 단위로 padding 처리. 가장 token 수가 많은 것에 맞춰서 padding ㄴ처리.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # max_length=10 : 10에 맞춤

# trainer 객체 생성 -> 학습/평가를 처리하는 객체
trainer = Trainer(
    model=model,
    args=train_args, # TrainingArguments
    train_dataset=trainset, # 학습데이터셋
    eval_dataset=testset, # 검증 데이터셋
    data_collator=data_collator,
    compute_metrics=compute_metrics #추가 평가/검증 함수
)

In [ ]:
# 학습
# trainer.train()

In [ ]:
trainer.evaluate() # eval_dataset으로 검증 

In [ ]:
# 모델을 hub에 업로드 하면 **토크나이저도 같이 업로드** 한다.
 #로그인
from huggingface_hub import login
login(hf_token)

tokenizer.push_to_hub(hub_model_id)

 # 학습 끝난 모델을 hub에 올리기
 # trainer.push_to_hub(모델_id)

In [61]:
# 모델/토크나이저 로딩
load_model = AutoModelForSequenceClassification.from_pretrained(
    hub_model_id,
    device_map="auto" # 실행 컴퓨터 환경에 맞춰 device를 설정
)
load_tokenizer = AutoTokenizer.from_pretrained(hub_model_id, device_map="auto")

config.json:   0%|          | 0.00/991 [00:00<?, ?B/s]

c:\Users\USER\Documents\SKN21\09_Huggingface_transformers\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--beth9165--youtube_comment_sentiment_kcbert_finetune. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' 

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

# 추론

In [62]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

txt = "재미있게 봤어요, 응원합니다."

# 1. 토큰화
tokens = load_tokenizer(txt, return_tensors="pt")
#tokens['input_ids'].device #실행되는 디바이스 확인
# 입력 데이터들의 device를 device로 설정
tokens = {key:value.to(device) for key, value in tokens.items()} 

# 추론
pred = load_model(**tokens)
pred_label = pred.logits.argmax(dim=-1).item()
pred_label

0

In [63]:
# 추론 함수

@torch.no_grad()
def predict(*docs):
  for doc in docs:
    tokens = load_tokenizer(doc, return_tensors="pt")
    tokens = {key:value.to(device) for key, value in tokens.items()}
    pred = load_model(**tokens)
    pred_proba = pred.logits.softmax(dim=-1) # 확률
    pred_proba = pred_proba.max(dim=-1).values
    pred_label = pred.logits.argmax(dim=-1).item()

    print(f"{doc} - {trainset['labels'].features.int2str(pred_label)}, {pred_proba}")

In [64]:
predict("와 제가 본 영상중에 제일 이해하기 쉽게 잘 설명 해둔 영상이네요!감사합니다!", "맨날 클릭하래")

와 제가 본 영상중에 제일 이해하기 쉽게 잘 설명 해둔 영상이네요!감사합니다! - 긍정, tensor([0.9995])
맨날 클릭하래 - 부정, tensor([0.9861])
